In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

import os

# 1. 이전까지의 방법으로 데이터 전처리

In [ ]:
# 데이터 불러오기
train = pd.read_csv('./kakr-4th-competition/train.csv')
test = pd.read_csv('./kakr-4th-competition/test.csv')
sample_submission = pd.read_csv('./kakr-4th-competition/sample_submission.csv')

In [ ]:
# 1) column 제거
def col_reduction(df):
    df.drop(['id','fnlwgt','education','relationship','native_country','workclass'], axis=1, inplace=True)
    
    return df

# 2) marital_status 조정
def mar_st(df):
    df['marital_status'] = (df['marital_status'] == 'Married-civ-spouse').astype(int)
    
    return df

# 3) capital_gain, loss 조정
def capital(df):
    df['cap_gain_high'] = (df['capital_gain'] != 0).astype(int)
    df['cap_loss_high'] = (df['capital_loss'] >= 1700).astype(int)
    df['capital_gain'] = df['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    
    return df

# 4) age 조정 함수
def age(df):
    df.loc[df['age'] < 20, 'age_range'] = '~20'
    df.loc[df['age'] >= 65, 'age_range'] = '~65'

    down = 20
    for i in range(45//5):
        df.loc[(df['age'] >= down) & (df['age'] < down+5), 'age_range'] = str(down)+'~'+str(down+5)
        down += 5

    df['age'] = df['age_range']
    df.drop(['age_range'], axis=1, inplace=True)
    
    return df
    
# 5) One-hot encoding은 만들지 않았다.

# 6) edu_num 새 변수 만들기
def edu(df):
    df['edu_num_high'] = (df['education_num'] >= 13).astype(int)
    
    return df

# 7) hpw 새 변수 만들기
    
def hpw(df):
    df['hpw_high'] = (df['hours_per_week'] >= 50).astype(int)

    return df

# 8) MinMaxScaler
def mm_feature(df, feature):
    mm_scaler = MinMaxScaler()
    
    df[feature] = mm_scaler.fit_transform(df[feature].values.reshape(-1,1))
    
    return df, mm_scaler

# 9) target 분리: train은 하고, test는 안하므로 따로 만들겠다.
def target_handle(df):
    df['income'] = df['income_>50K']
    df.drop(['income_>50K','income_<=50K'], axis=1, inplace=True)
    
    y_df = df.income
    X_df = df.drop(['income'], axis=1, inplace=False)
    
    return X_df, y_df

def main(df):
    
    df1 = col_reduction(df)
    df2 = mar_st(df1)
    df3 = capital(df2)
    df4 = age(df3)
    
    df5 = pd.get_dummies(df4)
    
    df6 = edu(df5)
    df_fin = hpw(df6)
    
    return df_fin

In [ ]:
# 적용
## main: 1) ~ 7)
train = main(train)
X_test = main(test)

## 8) minmax scaler
train, mm_scaler1 = mm_feature(train,'education_num')
train, mm_scaler2 = mm_feature(train,'hours_per_week')

X_test['education_num'] = mm_scaler1.transform(X_test['education_num'].values.reshape(-1,1))
X_test['hours_per_week'] = mm_scaler2.transform(X_test['hours_per_week'].values.reshape(-1,1))

## 9) X, y split
X_train, y_train = target_handle(train)

# 2. 모델링 (+KFold (k = 5))
1. Linear SVM
    - penalty: 'l1', '12'
    - loss: 'square hinge', 'hinge'
    1. 'l2', 'square hinge': 0.85024
    2. '12', 'hinge': 0.8436
2. kernel = 'rbf'
    - gamma: 'scale', 'auto', float
    1. 'scale': 0.77286
    2. 'auto': 0.8560800000000001
3. kernel = 'poly'
    - gamma: 'scale', 'auto'
    - degree
    - coefficient
    1. 'scale': 0.77386
    2. 'auto': 
4. kernel = 'sigmoid'
    - gamma: 'scale', 'auto'
    - coefficient
    1. 'scale': 0.7591800000000001
    2. 'auto': 0.77242

In [ ]:
svm_clf1 = LinearSVC()

# k = 5인 KFold와 Fold별 정확도를 담을 list 생성
kfold = KFold(n_splits=5) # default = 3
cv_accuracy = [] # 예측 성능을 list에 담을 것이다.
print('데이터 세트 크기:',X_train.shape[0])

n_iter = 0

# KFold.split( ) 호출: Fold 별 학습, 검증 Data의 row index를 array로 반환  
for train_index, test_index in kfold.split(X_train):
    
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train_, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_, y_val = y_train[train_index], y_train[test_index]
    
    # 학습 및 예측 
    svm_clf1.fit(X_train_ , y_train_)    
    pred = svm_clf1.predict(X_val)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(f1_score(y_val, pred, average='micro'), 4)
    train_size = X_train_.shape[0]
    test_size = X_val.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

데이터 세트 크기: 26049

#1 교차 검증 정확도 :0.8551, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#2 교차 검증 정확도 :0.8514, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#3 교차 검증 정확도 :0.8653, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#4 교차 검증 정확도 :0.8445, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#5 교차 검증 정확도 :0.8349, 학습 데이터 크기: 20840, 검증 데이터 크기: 5209

## 평균 검증 정확도: 0.85024


In [ ]:
svm_clf2 = SVC(gamma = 'auto')

# k = 5인 KFold와 Fold별 정확도를 담을 list 생성
kfold = KFold(n_splits=5) # default = 3
cv_accuracy = [] # 예측 성능을 list에 담을 것이다.
print('데이터 세트 크기:',X_train.shape[0])

n_iter = 0

# KFold.split( ) 호출: Fold 별 학습, 검증 Data의 row index를 array로 반환  
for train_index, test_index in kfold.split(X_train):
    
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train_, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_, y_val = y_train[train_index], y_train[test_index]
    
    # 학습 및 예측 
    svm_clf2.fit(X_train_ , y_train_)    
    pred = svm_clf2.predict(X_val)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(f1_score(y_val, pred, average='micro'), 4)
    train_size = X_train_.shape[0]
    test_size = X_val.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

데이터 세트 크기: 26049

#1 교차 검증 정확도 :0.8582, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#2 교차 검증 정확도 :0.847, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#3 교차 검증 정확도 :0.8706, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#4 교차 검증 정확도 :0.8522, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#5 교차 검증 정확도 :0.8524, 학습 데이터 크기: 20840, 검증 데이터 크기: 5209

## 평균 검증 정확도: 0.8560800000000001


In [ ]:
svm_clf3 = SVC(kernel = 'poly', gamma = 'auto')

# k = 5인 KFold와 Fold별 정확도를 담을 list 생성
kfold = KFold(n_splits=5) # default = 3
cv_accuracy = [] # 예측 성능을 list에 담을 것이다.
print('데이터 세트 크기:',X_train.shape[0])

n_iter = 0

# KFold.split( ) 호출: Fold 별 학습, 검증 Data의 row index를 array로 반환  
for train_index, test_index in kfold.split(X_train):
    
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train_, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_, y_val = y_train[train_index], y_train[test_index]
    
    # 학습 및 예측 
    svm_clf3.fit(X_train_ , y_train_)    
    pred = svm_clf3.predict(X_val)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(f1_score(y_val, pred, average='micro'), 4)
    train_size = X_train_.shape[0]
    test_size = X_val.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

데이터 세트 크기: 26049


In [ ]:
svm_clf4 = SVC(kernel = 'sigmoid', gamma = 'auto')

# k = 5인 KFold와 Fold별 정확도를 담을 list 생성
kfold = KFold(n_splits=5) # default = 3
cv_accuracy = [] # 예측 성능을 list에 담을 것이다.
print('데이터 세트 크기:',X_train.shape[0])

n_iter = 0

# KFold.split( ) 호출: Fold 별 학습, 검증 Data의 row index를 array로 반환  
for train_index, test_index in kfold.split(X_train):
    
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train_, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_, y_val = y_train[train_index], y_train[test_index]
    
    # 학습 및 예측 
    svm_clf4.fit(X_train_ , y_train_)    
    pred = svm_clf4.predict(X_val)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(f1_score(y_val, pred, average='micro'), 4)
    train_size = X_train_.shape[0]
    test_size = X_val.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

데이터 세트 크기: 26049

#1 교차 검증 정확도 :0.777, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#2 교차 검증 정확도 :0.7697, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#3 교차 검증 정확도 :0.7864, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#4 교차 검증 정확도 :0.7624, 학습 데이터 크기: 20839, 검증 데이터 크기: 5210

#5 교차 검증 정확도 :0.7666, 학습 데이터 크기: 20840, 검증 데이터 크기: 5209

## 평균 검증 정확도: 0.77242


In [ ]:
y_test_predict = svm_clf2.predict(X_test).astype(int)

sample_submission['prediction'] = y_test_predict
sample_submission.to_csv('submission4.csv', index=False)

# Test f1-score: 0.85682